In [1]:
!rm -rf gym-duckietown/
branch = "daffy" #@param ['master', 'daffy']
! git clone --branch {branch} https://github.com/duckietown/gym-duckietown
!pip install  gym-duckietown/.

Cloning into 'gym-duckietown'...
remote: Enumerating objects: 6116, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 6116 (delta 193), reused 314 (delta 137), pack-reused 5719
Receiving objects: 100% (6116/6116), 79.15 MiB | 16.37 MiB/s, done.
Resolving deltas: 100% (3502/3502), done.
Processing ./gym-duckietown
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 55.7 MB 1.3 MB/s 
     |████████████████████████████████| 48 kB 5.1 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 46 kB 3.5 

In [2]:
!pip install pyvirtualdisplay
!pip install piglet
!apt-get install python-opengl -y
!apt install xvfb -y
!apt-get install x11-utils
!pip install -U "ray[tune]"  
!pip install -U "ray[rllib]"  
!pip install -U "ray[serve]"
!pip install gym==0.22.0

     |████████████████████████████████| 67 kB 3.2 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 1s (576 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 155335 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed,

In [3]:
from pyvirtualdisplay import Display
import matplotlib.pyplot as plt
%matplotlib inline

import os
def create_display():
    display = Display(visible=False, size=(1400, 900))
    d = display.start()
    if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
        !bash ../xvfb start
        %env DISPLAY=:0
    return d

display = create_display()



In [4]:
import sys
import ray
from ray import tune
import gym_duckietown
from gym_duckietown.envs import DuckietownEnv
from gym_duckietown.simulator import Simulator, DEFAULT_ROBOT_SPEED, DEFAULT_CAMERA_WIDTH, DEFAULT_CAMERA_HEIGHT, CAMERA_FOV_Y, NotInLane
import gym
import torch
import multiprocessing
import pyglet
from collections import deque
from torch import nn
from torch import optim
import numpy as np
import random
from ray.tune import register_env
from ray.rllib.agents.ppo import ppo
from ray.rllib.agents.ppo import PPOTrainer
import random
import yaml
import glob
from ray.tune.logger import pretty_print
from pprint import pprint
import cv2
import logging
from gym import spaces
from matplotlib import pyplot as plt
import seaborn
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import norm

DEBUG:commons:version: 6.2.4 *
DEBUG:typing:version: 6.2.3
DEBUG:duckietown_world:duckietown-world version 6.2.38 path /usr/local/lib/python3.7/dist-packages
DEBUG:geometry:PyGeometry-z6 version 2.1.4 path /usr/local/lib/python3.7/dist-packages
DEBUG:aido_schemas:aido-protocols version 6.0.59 path /usr/local/lib/python3.7/dist-packages
DEBUG:nodes:version 6.2.13 path /usr/local/lib/python3.7/dist-packages pyparsing 3.0.7
DEBUG:gym-duckietown:gym-duckietown version 6.1.31 path /usr/local/lib/python3.7/dist-packages



{'audio': ('directsound', 'openal', 'pulse', 'silent'), 'debug_font': False, 'debug_gl': True, 'debug_gl_trace': False, 'debug_gl_trace_args': False, 'debug_graphics_batch': False, 'debug_lib': False, 'debug_media': False, 'debug_texture': False, 'debug_trace': False, 'debug_trace_args': False, 'debug_trace_depth': 1, 'debug_trace_flush': True, 'debug_win32': False, 'debug_x11': False, 'graphics_vbo': True, 'shadow_window': True, 'vsync': None, 'xsync': True, 'xlib_fullscreen_override_redirect': False, 'darwin_cocoa': True, 'search_local_libs': True}


In [26]:
logger = logging.getLogger(__name__)


def load_config(path: str = "./main.yaml"):
    # create base config
    config: dict = dict()
    with open(path) as file:
        config = yaml.load(file, yaml.Loader)
    # add sub-configs
    return config


def get_rllib_config(path):
    conf = load_config(path)["rllib_config"]
    conf["env_config"] = env_config(path)
    return conf


def env_config(path):
    conf = load_config(path)
    return conf["env_config"]


def get_default_rllib_conf():
    return ppo.DEFAULT_CONFIG.copy()

def add_env_conf(conf, new_conf):
    conf["env_config"] = new_conf
    return conf

def update_conf(conf):
    ray_conf = ppo.DEFAULT_CONFIG.copy()
    for key in conf:
        if key in ray_conf:
            ray_conf[key] = conf[key]
    return ray_conf

In [6]:
class Heading2WheelVelsWrapper(gym.ActionWrapper):
    def __init__(self, env, heading_type=None):
        super(Heading2WheelVelsWrapper, self).__init__(env)
        self.heading_type = heading_type
        self.action_space = spaces.Box(low=-1., high=1., shape=(1,))
        if self.heading_type == 'heading_trapz':
            straight_plateau_half_width = 0.3333  # equal interval for left, right turning and straight
            self.mul = 1. / (1. - straight_plateau_half_width)

    def action(self, action):
        if isinstance(action, tuple):
            action = action[0]
        # action = [-0.5 * action + 0.5, 0.5 * action + 0.5]
        if self.heading_type == 'heading_smooth':
            action = np.clip(np.array([1 + action ** 3, 1 - action ** 3]), 0., 1.)  # Full speed single value control
        elif self.heading_type == 'heading_trapz':
            action = np.clip(np.array([1 - action, 1 + action]) * self.mul, 0., 1.)
        elif self.heading_type == 'heading_sine':
            action = np.clip([1 - np.sin(action * np.pi), 1 + np.sin(action * np.pi)], 0., 1.)
        elif self.heading_type == 'heading_limited':
            action = np.clip(np.array([1 + action*0.666666, 1 - action*0.666666]), 0., 1.)
        else:
            action = np.clip(np.array([1 + action, 1 - action]), 0., 1.)  # Full speed single value control
        return action

In [7]:
!pwd

/content


In [8]:
class ClipImageWrapper(gym.ObservationWrapper):
    def __init__(self, env=None, top_margin_divider=3):
        super(ClipImageWrapper, self).__init__(env)
        img_height, img_width, depth = self.observation_space.shape
        top_margin = img_height // top_margin_divider
        img_height = img_height - top_margin
        # Region Of Interest
        # r = [margin_left, margin_top, width, height]
        self.roi = [0, top_margin, img_width, img_height]

        self.observation_space = spaces.Box(
            self.observation_space.low[0, 0, 0],
            self.observation_space.high[0, 0, 0],
            (img_height, img_width, depth),
            dtype=self.observation_space.dtype)

    def observation(self, observation):
        r = self.roi
        observation = observation[int(r[1]):int(r[1] + r[3]), int(r[0]):int(r[0] + r[2])]
        return observation

In [9]:
class ResizeWrapper(gym.ObservationWrapper):
    def __init__(self, env=None, shape=(120, 160)):
        super(ResizeWrapper, self).__init__(env)
        if isinstance(shape, str):
            self.shape = eval(shape) + (self.observation_space.shape[2],)  # Depth is unchanged and can have any value
        else:
            self.shape = shape + (self.observation_space.shape[2],)  # Depth is unchanged and can have any value
        self.observation_space = spaces.Box(
            self.observation_space.low[0, 0, 0],
            self.observation_space.high[0, 0, 0],
            self.shape,
            dtype=self.observation_space.dtype)

    def observation(self, observation):
        resized = cv2.resize(observation, self.shape[:2][::-1], interpolation=cv2.INTER_AREA, )
        if len(resized.shape) == 2:
            resized = np.expand_dims(resized, 2)
        return resized

In [10]:
class MotionBlurWrapper(gym.ObservationWrapper):
    """
    Simulates motion blur separately for horizontal (left-right) rotational and forward movement.
    Both are simulated using a smoothing convolutional filter (with varying sized "horizontal" or "vertical" kernel).
    Forward motion should produce non-uniform blur (none at the center point of the horizon, and larger towards the
     edges), here it is simulated using the same filtering over the whole image (the size of the kernel depends on the
     robot velocity.

    According to gluPerspective (used by the Simulator) x is width, y is height.
    """

    def __init__(self, env):
        super(MotionBlurWrapper, self).__init__(env)
        self.camera_fov_height_angle = self.unwrapped.cam_fov_y / 180. * np.pi
        self.camera_fov_width_angle = \
            self.unwrapped.camera_width / float(self.unwrapped.camera_height) * self.camera_fov_height_angle
        self.camera_height_px = self.observation_space.shape[0]
        self.camera_width_px = self.observation_space.shape[1]
        self.blur_time = 0.05
        self.prev_angle = self.unwrapped.cur_angle
        self.simulate_rotational_blur = True
        self.simulate_forward_blur = False  # Vertical convolution doesn't work well for this

    def observation(self, observation):
        if self.simulate_rotational_blur:
            cur_angle = self.unwrapped.cur_angle
            angular_vel = self._angle_diff(self.prev_angle,
                                           cur_angle) * self.unwrapped.frame_rate * self.unwrapped.frame_skip
            self.prev_angle = cur_angle
            delta_angle = angular_vel * self.blur_time
            if abs(delta_angle) > 0:
                ksize = np.round(np.abs(delta_angle / self.camera_fov_width_angle * self.camera_width_px)).astype(
                    int) + 1
                logger.debug("Rotational motion blur kernel size {}".format(ksize))
                kernel = np.zeros((ksize, ksize))
                kernel[ksize // 2, :] = 1. / ksize
                observation = cv2.filter2D(observation, -1, kernel)
        if self.simulate_forward_blur:
            # Empirical kernel size, proportional to the current speed over the max speed of the robot
            ksize = np.round(
                np.abs(self.camera_width_px / 30 * self.unwrapped.speed / self.unwrapped.robot_speed)).astype(int) + 1
            logger.debug("Forward motion blur kernel size {}".format(ksize))
            kernel = np.zeros((ksize, ksize))
            kernel[:, ksize // 2] = 1. / ksize
            observation = cv2.filter2D(observation, -1, kernel)
        return observation

In [11]:
class NormalizeWrapper(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(NormalizeWrapper, self).__init__(env)
        self.obs_lo = self.observation_space.low[0, 0, 0]
        self.obs_hi = self.observation_space.high[0, 0, 0]
        obs_shape = self.observation_space.shape
        self.observation_space = spaces.Box(0.0, 1.0, obs_shape, dtype=np.float32)

    def observation(self, obs):
        if self.obs_lo == 0.0 and self.obs_hi == 1.0:
            return obs
        else:
            return (obs - self.obs_lo) / (self.obs_hi - self.obs_lo)

In [34]:
from pprint import pprint
logger = logging.getLogger(__name__)


class Environment:
    def __init__(self, seed):
        self._env = None
        np.random.seed(seed)
        random.seed(seed)

    def create_env(self, env_config, env_id=0) -> Simulator:
        pprint(env_config)
        self._env = Simulator(
            seed=env_config["seed"],
            map_name=env_config["map_name"],
            max_steps=env_config["max_steps"],
            camera_width=env_config["camera_width"],
            camera_height=env_config["camera_height"],
            accept_start_angle_deg=env_config["accept_start_angle_deg"],
            full_transparency=env_config["full_transparency"],
            distortion=env_config["distortion"],
            domain_rand=env_config["domain_rand"],
        )
        self._wrap()
        return self._env

    def _wrap(self) -> None:
        #self._env = LastPictureObsWrapper(self._env)
        #self._env = ActionDelayWrapper(self._env)
        self._env = ClipImageWrapper(self._env, 3)
        #self._env = RGB2GrayscaleWrapper(self._env)
        self._env = ResizeWrapper(self._env, (84, 84))
        #self._env = RandomFrameRepeatingWrapper(self._env)
        #self._env = ObservationBufferWrapper(self._env)
        self._env = MotionBlurWrapper(self._env)
        self._env = NormalizeWrapper(self._env)
        self._env = Heading2WheelVelsWrapper(self._env)
        #self._env = ActionSmoothingWrapper(self._env)
        #self._env = DtRewardTargetOrientation(self._env)
        #self._env = DtRewardVelocity(self._env)
        #self._env = DtRewardCollisionAvoidance(self._env)


In [37]:

ray.shutdown()

from pyvirtualdisplay import Display
import matplotlib.pyplot as plt
%matplotlib inline

import os
def create_display():
    display = Display(visible=False, size=(1400, 900))
    d = display.start()
    if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
        !bash ../xvfb start
        %env DISPLAY=:0
    return d

display = create_display()

!Xvfb :0 -screen 0 1024x768x24 -ac +extension GLX +render -noreset &> xvfb.log &
!export DISPLAY=:0

ENV_NAME = 'Duckietown'
ray_init_config = {
    "num_cpus": multiprocessing.cpu_count(),
    "num_gpus": torch.cuda.device_count(),
    "ignore_reinit_error": True,
}

env = Environment(random.randint(0, 100000))

ray.init(**ray_init_config)

register_env(ENV_NAME, env.create_env)

rllib_config = get_default_rllib_conf()
rllib_config.update({
    "env": ENV_NAME,
    "num_gpus": torch.cuda.device_count(),
    "num_workers": multiprocessing.cpu_count()-1,
    "gpus_per_worker": torch.cuda.device_count(),
    "env_per_worker": 1,
    "framework": "torch",
    "lr": 0.0001,
})

conf = update_conf(rllib_config)
conf = add_env_conf(conf, {
    "seed": random.randint(0, 100000),
    "map_name": "loop_empty",
    "max_steps": 5000,
    "camera_width": 640,
    "camera_height": 480,
    "accept_start_angle_deg": 40,
    "full_transparency": True,
    "distortion": True,
    "domain_rand": False
})
pprint(conf)
tune.run(
    PPOTrainer,
    stop={"timesteps_total": 2000000},
    checkpoint_at_end=True,
    config=conf,
    trial_name_creator=lambda trial: trial.trainable_name,
    checkpoint_freq=1,
    keep_checkpoints_num=2,
    checkpoint_score_attr="episode_reward_mean"
)

2022-03-12 12:02:30,194	INFO services.py:1414 -- View the Ray dashboard at http://127.0.0.1:8266


{'_disable_action_flattening': False,
 '_disable_execution_plan_api': False,
 '_disable_preprocessor_api': False,
 '_fake_gpus': False,
 '_tf_policy_handles_more_than_one_loss': False,
 'action_space': None,
 'actions_in_input_normalized': False,
 'always_attach_evaluation_results': False,
 'batch_mode': 'truncate_episodes',
 'callbacks': <class 'ray.rllib.agents.callbacks.DefaultCallbacks'>,
 'clip_actions': False,
 'clip_param': 0.3,
 'clip_rewards': None,
 'collect_metrics_timeout': -1,
 'compress_observations': False,
 'create_env_on_driver': False,
 'custom_eval_function': None,
 'custom_resources_per_worker': {},
 'eager_max_retraces': 20,
 'eager_tracing': False,
 'entropy_coeff': 0.0,
 'entropy_coeff_schedule': None,
 'env': 'Duckietown',
 'env_config': {'accept_start_angle_deg': 40,
                'camera_height': 480,
                'camera_width': 640,
                'distortion': True,
                'domain_rand': False,
                'full_transparency': True,
     

Trial name,status,loc
PPOTrainer,PENDING,


Trial name,status,loc
PPOTrainer,PENDING,


Trial name,status,loc
PPOTrainer,PENDING,


Trial name,status,loc
PPOTrainer,PENDING,


2022-03-12 12:02:50,863	WARNING tune.py:596 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc
PPOTrainer,PENDING,


2022-03-12 12:02:51,085	ERROR tune.py:635 -- Trials did not complete: [PPOTrainer]
2022-03-12 12:02:51,087	INFO tune.py:639 -- Total run time: 17.01 seconds (16.78 seconds for the tuning loop).
2022-03-12 12:02:51,090	WARNING tune.py:644 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [ ]:
!nvidia-smi

In [ ]:
config = {
    "seed": random.randint(0, 100000),
    "map_name": "loop_empty",
    "max_steps": 5000,
    "camera_width": 640,
    "camera_height": 480,
    "accept_start_angle_deg": 40,
    "full_transparency": True,
    "distortion": True,
    "domain_rand": False
}
env = Environment(config, random.randint(0, 100000))
env1 = env.create_env(None)

In [ ]:
!ps -a